<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=b4cb1de8b636f0112c49b9a596fa9d72dc7d72b0c58a5ed97c029b18e6692337
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.2
    Uninstalling protobuf-6.33.2:
      Successfully uninstalled protobuf-6.33.2


  Attempting uninstall: yfinance
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-06 16:12:05
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.44 C
-------------------
Today PnL: 17.26 K (0.12%)
Current PnL: -24.77 L (-15.8%)
CY Booked + Current PnL: -10.48 L (-6.68%)
-------------------
Total profit:  2.04 L
Total loss:  -26.80 L
-------------------
Total Booked + Current PnL: 16.50 L (12.71%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (9.9%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.34 C
Est FTT PnL: 89.74 L (62.18%)
Deployed:  1.30 C
Current:  1.44 C
CAGR/XIRR %: 5.81%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
82,VOLTAS,1530.00,0.64,16.27,2.95,19.69,6576.0,31188.0,222930.0,4.86,71.0,X-MC,3.60,78.0,4.74,1.54,24.17,XY25,NTT,AC
51,NESTLEIND,1377.00,0.40,16.36,4.33,21.40,13374.0,43431.0,308857.0,7.98,71.0,X-LC,5.02,12.0,3.25,2.14,23.73,XY25,NTT,FMCG
49,MASFIN,398.61,5.47,3.52,17.90,22.05,18156.0,3450.0,101430.0,-10.57,64.0,H-SC,6.62,164.0,0.19,0.70,47.44,XR,ATH,FINANCE
77,TTKPRESTIG,770.00,-1.26,-19.66,24.59,0.09,19909.0,-19812.0,80965.0,87.83,42.0,M-SC,3.28,253.0,-1.00,0.56,6.37,OX40N,NTT,DURABLES
17,COALINDIA,484.83,-0.09,5.02,13.53,19.23,21378.0,7548.0,158008.0,22.81,82.0,L-LC,5.65,179.0,0.35,1.10,27.36,XY25,ATH,MINING


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,RAJOOENG,143.10,11.64,-24.20,84.17,39.61,65400.0,-24800.0,77700.0,-48.73,69.0,H-SC,32.17,135.0,-0.38,0.54,25.77,AR,ATH,MISC
37,IEX,219.00,10.63,0.90,47.31,48.63,95446.0,1791.0,201745.0,-32.92,66.0,H-SC,14.30,136.0,0.02,1.40,12.62,XR,NTT,MISC
49,MASFIN,398.61,5.47,3.52,17.90,22.05,18156.0,3450.0,101430.0,-10.57,64.0,H-SC,6.62,164.0,0.19,0.70,47.44,XR,ATH,FINANCE
31,HAVELLS,2069.16,3.47,-3.85,37.79,32.48,119173.0,-12634.0,315357.0,-8.67,71.0,X-MC,6.05,67.0,-0.11,2.19,8.04,X40,ATH,ELECTRICAL
0,5PAISA,593.00,3.29,-26.14,52.85,12.90,85293.0,-57109.0,161387.0,137.98,73.0,H-SC,10.97,161.0,-0.67,1.12,34.19,OX40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
59,RELIANCE,1952.00,-4.47,-3.23,29.48,25.30,57333.0,-6483.0,194480.0,-6.52,39.0,X-LC,8.97,17.0,-0.11,1.35,30.27,XY25,BTT,REFINERIES
83,WHIRLPOOL,2270.00,-4.05,-29.60,158.41,81.92,128022.0,-33981.0,80817.0,-58.80,31.0,M-SC,24.75,236.0,-0.27,0.56,0.00,XR,NTT,DURABLES
74,TITAGARH,1548.00,-2.71,-19.09,80.13,45.74,157009.0,-46234.0,195943.0,7.85,53.0,H-SC,2.98,174.0,-0.29,1.36,27.44,XY24,NTT,ENGINEERING
22,DIXON,18931.72,-2.70,-18.70,61.45,31.26,115290.0,-43152.0,187616.0,-62.00,28.0,X-MC,18.04,56.0,-0.37,1.30,0.00,X40N,ATH,IT
50,MEDANTA,1486.00,-2.31,0.94,22.92,24.08,29370.0,1193.0,128143.0,-7.25,54.0,X-SC,7.03,89.0,0.04,0.89,20.74,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.01,-0.97,109.94,107.91,160749.0,-1429.0,146215.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.02,OX40N,ATH,CABLES
37,IEX,219.00,10.63,0.90,47.31,48.63,95446.0,1791.0,201745.0,-32.92,66.0,H-SC,14.30,136.0,0.02,1.40,12.62,XR,NTT,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.01,-0.97,109.94,107.91,160749.0,-1429.0,146215.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.02,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.48,-15.43,18.27,0.02,26971.0,-26937.0,147622.0,124.89,52.0,M-SC,7.24,234.0,-1.00,1.02,28.41,OX40N,NTT,PAINTS
77,TTKPRESTIG,770.00,-1.26,-19.66,24.59,0.09,19909.0,-19812.0,80965.0,87.83,42.0,M-SC,3.28,253.0,-1.00,0.56,6.37,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.39,-19.82,41.52,13.47,89777.0,-53442.0,216225.0,-66.95,59.0,H-SC,5.29,158.0,-0.60,1.50,9.94,XY24,NTT,PAINTS
66,SIS,528.00,-0.10,-22.39,56.24,21.26,48466.0,-24855.0,86177.0,2035.23,57.0,H-SC,4.28,166.0,-0.51,0.60,16.43,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,0.49,3.64,16.49,20.74,34723.0,7398.0,210568.0,35.83,74.0,M-MC,6.85,189.0,0.21,1.46,14.64,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.09,5.02,13.53,19.23,21378.0,7548.0,158008.0,22.81,82.0,L-LC,5.65,179.0,0.35,1.10,27.36,XY25,ATH,MINING
84,WIPRO,420.00,0.87,8.94,58.13,72.27,107303.0,15144.0,184592.0,-6.53,57.0,M-LC,5.82,99.0,0.14,1.28,15.66,XR,NTT,IT
49,MASFIN,398.61,5.47,3.52,17.90,22.05,18156.0,3450.0,101430.0,-10.57,64.0,H-SC,6.62,164.0,0.19,0.70,47.44,XR,ATH,FINANCE
37,IEX,219.00,10.63,0.90,47.31,48.63,95446.0,1791.0,201745.0,-32.92,66.0,H-SC,14.30,136.0,0.02,1.40,12.62,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,MASFIN,398.61,5.47,3.52,17.90,22.05,18156.0,3450.0,101430.0,-10.57,64.0,H-SC,6.62,164.0,0.19,0.70,47.44,XR,ATH,FINANCE
84,WIPRO,420.00,0.87,8.94,58.13,72.27,107303.0,15144.0,184592.0,-6.53,57.0,M-LC,5.82,99.0,0.14,1.28,15.66,XR,NTT,IT
37,IEX,219.00,10.63,0.90,47.31,48.63,95446.0,1791.0,201745.0,-32.92,66.0,H-SC,14.30,136.0,0.02,1.40,12.62,XR,NTT,MISC
25,FINCABLES,1641.55,-0.01,-0.97,109.94,107.91,160749.0,-1429.0,146215.0,-19.34,56.0,M-SC,10.23,216.0,-0.01,1.01,9.02,OX40N,ATH,CABLES
85,ZYDUSLIFE,1286.17,0.99,-1.82,38.14,35.63,78478.0,-3812.0,205762.0,-15.85,55.0,H-MC,2.95,119.0,-0.05,1.43,14.55,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,-2.07,-16.07,31.99,10.78,64525.0,-38615.0,201703.0,-50.14,13.0,X-LC,4.26,5.0,-0.60,1.40,0.00,X40,NTT,FMCG
22,DIXON,18931.72,-2.70,-18.70,61.45,31.26,115290.0,-43152.0,187616.0,-62.00,28.0,X-MC,18.04,56.0,-0.37,1.30,0.00,X40N,ATH,IT
7,AWL,485.00,-1.28,-25.20,110.14,57.19,268693.0,-82171.0,243956.0,-64.65,29.0,X-MC,8.12,58.0,-0.31,1.69,0.00,XY24,NTT,FMCG
23,DMART,5391.45,0.48,-11.93,47.16,29.60,103668.0,-29778.0,219822.0,-24.85,30.0,X-LC,9.69,19.0,-0.29,1.52,7.63,X40N,ATH,FMCG
52,PAGEIND,51605.07,-0.53,-10.50,45.94,30.62,97466.0,-24894.0,212160.0,-36.35,33.0,X-MC,15.85,55.0,-0.26,1.47,0.00,X40,ATH,APPARELS


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.0,0.08,1.12,19.42,20.77,45982.0,2631.0,236775.0,-7.61,39.0,X-MC,0.95,62.0,0.06,1.64,8.12,X40N,NTT,BREWERIES
65,SIEMENS,4671.5,1.34,-15.95,49.34,25.51,77170.0,-29690.0,156405.0,-3.93,50.0,H-LC,1.34,49.0,-0.38,1.08,15.26,AR,ATH,ELECTRICAL
20,DABUR,735.0,-0.16,1.73,41.25,43.70,104103.0,4302.0,252370.0,-6.68,66.0,X-MC,1.95,72.0,0.04,1.75,17.05,XY24,BTT,FMCG
12,BERGEPAINT,680.0,-0.34,-6.15,29.08,21.14,62043.0,-13985.0,213354.0,-12.71,40.0,X-MC,2.27,74.0,-0.23,1.48,18.48,XY24,NTT,PAINTS
45,JIOFIN,387.0,-0.62,-3.97,29.63,24.48,76076.0,-10612.0,256753.0,-5.55,50.0,X-LC,2.30,36.0,-0.14,1.78,48.82,XY24,BTT,FINANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
7,AWL,485.00,-1.28,-25.20,110.14,57.19,268693.0,-82171.0,243956.0,-64.65,29.0,X-MC,8.12,58.0,-0.31,1.69,0.00,XY24,NTT,FMCG
22,DIXON,18931.72,-2.70,-18.70,61.45,31.26,115290.0,-43152.0,187616.0,-62.00,28.0,X-MC,18.04,56.0,-0.37,1.30,0.00,X40N,ATH,IT
52,PAGEIND,51605.07,-0.53,-10.50,45.94,30.62,97466.0,-24894.0,212160.0,-36.35,33.0,X-MC,15.85,55.0,-0.26,1.47,0.00,X40,ATH,APPARELS
43,ITC,452.00,-2.07,-16.07,31.99,10.78,64525.0,-38615.0,201703.0,-50.14,13.0,X-LC,4.26,5.0,-0.60,1.40,0.00,X40,NTT,FMCG
11,BATAINDIA,2096.00,-1.17,-38.98,121.59,35.20,95460.0,-50160.0,78510.0,2.96,40.0,X-SC,17.24,92.0,-0.53,0.54,0.84,X40,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,2.66,-26.33,93.88,42.84,44962.0,-17113.0,47893.0,-51.52,57.0,X-SC,29.20,83.0,-0.38,0.33,8.35,XY24,NTT,MISC
58,RELAXO,1176.00,-0.47,-47.63,190.84,52.31,145073.0,-69142.0,76018.0,-44.25,46.0,X-SC,7.38,91.0,-0.48,0.53,1.75,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.17,-38.98,121.59,35.20,95460.0,-50160.0,78510.0,2.96,40.0,X-SC,17.24,92.0,-0.53,0.54,0.84,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,-2.31,0.94,22.92,24.08,29370.0,1193.0,128143.0,-7.25,54.0,X-SC,7.03,89.0,0.04,0.89,20.74,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.45,-16.07,70.76,43.32,96729.0,-26172.0,136700.0,-27.94,53.0,X-SC,7.77,90.0,-0.27,0.95,4.96,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
73,TCS,4389.97,1.23,-10.48,34.84,20.71,117969.0,-39634.0,338603.0,-22.98,55.0,X-LC,2.66,1.0,-0.34,2.35,13.14,X40,ATH,IT
41,INFY,1972.00,0.36,9.75,22.32,34.24,76647.0,30511.0,343399.0,-14.57,50.0,X-LC,7.46,2.0,0.40,2.38,18.84,X40,NTT,IT
75,TMPV,600.00,-1.24,-24.39,62.65,22.98,107007.0,-55097.0,170801.0,-21.86,58.0,X-LC,4.96,3.0,-0.51,1.18,7.43,XY24,NTT,AUTO
81,VBL,671.64,2.08,0.77,34.64,35.67,110247.0,2424.0,318266.0,-11.24,66.0,X-LC,2.36,4.0,0.02,2.21,14.73,X40N,ATH,FMCG
43,ITC,452.00,-2.07,-16.07,31.99,10.78,64525.0,-38615.0,201703.0,-50.14,13.0,X-LC,4.26,5.0,-0.60,1.40,0.00,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,1.59,-32.50,96.88,32.89,50486.0,-25094.0,52112.0,-743.79,73.0,L-MC,7.19,259.0,-0.50,0.36,43.51,XR,NTT,BANKS
49,MASFIN,398.61,5.47,3.52,17.90,22.05,18156.0,3450.0,101430.0,-10.57,64.0,H-SC,6.62,164.0,0.19,0.70,47.44,XR,ATH,FINANCE
13,BSOFT,831.70,-1.38,-23.62,97.41,50.79,104650.0,-33221.0,107432.0,-2.20,47.0,H-SC,10.68,151.0,-0.32,0.74,25.09,XR,ATH,IT
39,INDIGOPNTS,1408.00,0.48,-15.43,18.27,0.02,26971.0,-26937.0,147622.0,124.89,52.0,M-SC,7.24,234.0,-1.00,1.02,28.41,OX40N,NTT,PAINTS
69,SURYODAY,216.00,1.11,-17.81,49.70,23.03,73146.0,-31895.0,147176.0,60.09,58.0,H-SC,5.36,167.0,-0.44,1.02,46.03,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,3.29,-26.14,52.85,12.90,85293.0,-57109.0,161387.0,137.98,73.0,H-SC,10.97,161.0,-0.67,1.12,34.19,OX40N,NTT,FINANCE
56,RAJOOENG,143.10,11.64,-24.20,84.17,39.61,65400.0,-24800.0,77700.0,-48.73,69.0,H-SC,32.17,135.0,-0.38,0.54,25.77,AR,ATH,MISC
17,COALINDIA,484.83,-0.09,5.02,13.53,19.23,21378.0,7548.0,158008.0,22.81,82.0,L-LC,5.65,179.0,0.35,1.10,27.36,XY25,ATH,MINING
74,TITAGARH,1548.00,-2.71,-19.09,80.13,45.74,157009.0,-46234.0,195943.0,7.85,53.0,H-SC,2.98,174.0,-0.29,1.36,27.44,XY24,NTT,ENGINEERING
82,VOLTAS,1530.00,0.64,16.27,2.95,19.69,6576.0,31188.0,222930.0,4.86,71.0,X-MC,3.60,78.0,4.74,1.54,24.17,XY25,NTT,AC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.11
1,25,44.14
2,50,76.47


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.84
MC    30.90
LC    26.27
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.50
X40      23.86
X40N     12.71
XY25     10.72
XR        9.05
AR        8.67
OX40N     7.55
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.39
H-SC    22.95
X-LC    22.81
M-SC    11.27
X-SC     7.84
H-MC     4.69
M-MC     1.46
M-LC     1.28
L-LC     1.10
H-LC     1.08
L-SC     0.78
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.95,-7.68,43.01
IT,12.60,-17.58,76.89
FINANCE,11.18,-10.62,57.04
MISC,7.19,-25.13,76.32
ELECTRICAL,5.96,-9.23,49.24
PAINTS,5.74,-11.56,28.14
INSURANCE,4.74,0.74,33.23
PHARMA,3.99,-2.20,34.37
AUTO,2.83,-28.39,72.90


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3147059.0,21
AR,1321755.0,10
XR,1185353.0,12
X40,1080169.0,15
X40N,907627.0,9
OX40N,701203.0,10
XY25,346880.0,7
SR,283539.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3485098.0,24
X-MC,1465552.0,16
M-SC,1430755.0,15
X-LC,933876.0,13
X-SC,786809.0,8
H-MC,403403.0,3
L-SC,177032.0,2
M-LC,107303.0,1
H-LC,77170.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1226947.0      6
           AR         909375.0      5
M-SC       XY24       817577.0      6
H-SC       XR         760125.0      7
X-MC       X40        474547.0      7
           XY24       434839.0      3
X-LC       X40        416440.0      6
X-MC       X40N       385825.0      4
X-SC       X40N       307887.0      3
M-SC       OX40N      305691.0      5
H-SC       OX40N      305112.0      4
X-SC       XY24       289740.0      3
H-SC       SR         283539.0      2
X-LC       X40N       213915.0      2
H-MC       AR         208530.0      2
           XY24       194873.0      1
X-SC       X40        189182.0      2
X-LC       XY24       183083.0      2
M-SC       XR         180807.0      2
X-MC       XY25       170341.0      2
M-SC       AR         126680.0      2
X-LC       XY25       120438.0      3
M-LC       XR         107303.0      1
L-SC       OX40N       90400.0      1
           XR          86632.0      1
H-LC       AR          77170.0      1
L-MC       XR          50486.0      1
M-MC       XY25        34723.0      1
L-LC       XY25        21378.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
